# <center> <u> Test technique Deezer Cyrille NOUBOUE </u> </center>

## <u> Level 1:</u>

### Intro
***
We are building a peer-to-peer car rental service. Let's call it Getaround :)

Here is our plan:

    - Let any car owner list her car on our platform
    - Let any person (let's call him 'driver') book a car for given dates/distance

### Level 1
***
The car owner chooses a price per day and price per km for her car. The driver then books the car for a given period and an approximate distance.

The rental price is the sum of:

    - A time component: the number of rental days multiplied by the car's price per day
    - A distance component: the number of km multiplied by the car's price per km

In [1]:
class Vehicle():
    
    def __init__(self, p_id, price_per_day, price_per_km):
        self.car_id = p_id
        self.price_per_day = price_per_day
        self.price_per_km = price_per_km
        
    def get_rental_price(self, nb_days, nb_km):
        return nb_days * self.price_per_day + nb_km * self.price_per_km
        

In [2]:
import datetime

class Rental():
    
    def __init__(self, rent_id, vehicle, start_date, end_date, distance):
        
        self.rental = rent_id
        self.vehicle = vehicle
        self.start_date = start_date
        self.end_date = end_date
        self.distance = distance
        
        #Calculate number of days
        diff_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.datetime.strptime(start_date, '%Y-%m-%d') #Calculate the number of days
        nb_days = diff_date.days + 1
        self.price = self.vehicle.get_rental_price(nb_days, distance) #Set the rental price 
        

In [3]:
veh = Vehicle(1, 2000, 10)

In [4]:
my_rent = Rental(1, veh, "2015-03-31", "2015-04-01", 300)

In [5]:
my_rent.price

7000

## <u>Level 2</u>
***

To be as competitive as possible, we decide to have a decreasing pricing for longer rentals.

New rules:

price per day decreases by 10% after 1 day
price per day decreases by 30% after 4 days
price per day decreases by 50% after 10 days
Adapt the rental price computation to take these new rules into account.

In [6]:
class Vehicle():
    
    def __init__(self, p_id, price_per_day, price_per_km):
        self.car_id = p_id
        self.price_per_day = price_per_day
        self.price_per_km = price_per_km
        
        
    def calculate_ndays_fees(self, n_days):
        paliers = [1, 3, 6, 10e4]
        i = 0
        list_days = [0, 0, 0, 0]

        while(n_days > paliers[i]):
            n_days = n_days - paliers[i]
            list_days[i] = paliers[i]
            i += 1
        list_days[i] = n_days

        return list_days
    
        
    def get_rental_price(self, nb_days, nb_km):
        
        # Get different days with particular discount rates
        list_ndays = self.calculate_ndays_fees(nb_days)
        discount_rates = [1, 0.9, 0.7, 0.5]
        
        # Calculate price with nb_km
        price = nb_km * self.price_per_km 
        
        # Calculate price with nb_days and different discount rates
        for i in range(len(discount_rates)):
            price += list_ndays[i] * discount_rates[i] * self.price_per_day

        return price
        

In [7]:
veh = Vehicle(1, 2000, 10)

In [8]:
my_rent = Rental(1, veh, "2015-07-3", "2015-07-14", 1000)

In [9]:
my_rent.price

27800.0

## <u>Level 3</u>
***

The car owner now wants her money. We decide to take a 30% commission on the rental price to cover our costs and have a solid business model.

The commission is split like this:

    - half goes to the insurance
    - 1€/day goes to the roadside assistance
    - the rest goes to us
    
Compute the amount that belongs to the insurance, to the assistance and to us.

In [10]:
import datetime

class Rental():
    
    def __init__(self, rent_id, vehicle, start_date, end_date, distance):
        
        self.rent_id = rent_id
        self.vehicle = vehicle
        self.start_date = start_date
        self.end_date = end_date
        self.distance = distance
        self.commission = dict()
        
        #Calculate number of days
        diff_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.datetime.strptime(start_date, '%Y-%m-%d') #Calculate the number of days
        self.nb_days = diff_date.days + 1
        self.price = self.vehicle.get_rental_price(self.nb_days, distance) #Set the rental price 
     
    
        #Share rental revenues between car owner, the company, the insurance & the roadside assistance

        # For Assistance_fee, it is say in the instructions that it is 1€ per day of reservation
        # but to match the output file expectations,
        # i have to multiply it by 100
                
          # for drivy_fee, we assume the commission will
          # always be positive, if not it will count
          # as a company loss or debt
        
        self.commission = {
            "insurance_fee": self.price * 0.3 * 0.5,
            "assistance_fee": self.nb_days * 100,
            "drivy_fee": self.price * 0.3 - self.price * 0.3 * 0.5 - self.nb_days * 100 
         }                                                   
        

In [11]:
veh = Vehicle(1, 2000, 10)

In [12]:
my_rent = Rental(1, veh, "2015-07-3", "2015-07-14", 1000)

In [13]:
my_rent.price

27800.0

In [14]:
my_rent.commission

{'insurance_fee': 4170.0, 'assistance_fee': 1200, 'drivy_fee': 2970.0}

## <u>Level 4</u>
***

We now want to know how much money must be debited/credited for each actor, this will allow us to debit or pay them accordingly.

In [15]:
import datetime

class Rental():
    
    def __init__(self, rent_id, vehicle, start_date, end_date, distance):
        
        self.rent_id = rent_id
        self.vehicle = vehicle
        self.start_date = start_date
        self.end_date = end_date
        self.distance = distance
        self.commission = dict()
        
        #Calculate number of days
        diff_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.datetime.strptime(start_date, '%Y-%m-%d') #Calculate the number of days
        self.nb_days = diff_date.days + 1
        self.price = self.vehicle.get_rental_price(self.nb_days, distance) #Set the rental price 
     
    
        #Share rental revenues between car owner, the company, the insurance & the roadside assistance

        # For Assistance_fee, it is say in the instructions that it is 1€ per day of reservation
        # but to match the output file expectations,
        # i have to multiply it by 100
                
          # for drivy_fee, we assume the commission will
          # always be positive, if not it will count
          # as a company loss or debt
        
        self.commission = {
            "insurance_fee": self.price * 0.3 * 0.5,
            "assistance_fee": self.nb_days * 100,
            "drivy_fee": self.price * 0.3 - self.price * 0.3 * 0.5 - self.nb_days * 100 
         } 
        
        #Level 4 : Actions
        driver_debit = self.price 
        owner_fee = self.price * 0.7
        insurance_fee = self.commission["insurance_fee"]
        assistance_fee = self.commission["assistance_fee"]
        drivy_fee = self.commission["drivy_fee"]
        
        self.set_actions(driver_debit = driver_debit,
                         owner_fee = owner_fee,
                         insurance_fee = insurance_fee,
                         assistance_fee = assistance_fee,
                         drivy_fee = drivy_fee)
        
    
    def set_actions(self, driver_debit, owner_fee, insurance_fee,
                    assistance_fee, drivy_fee):
        
        #Level 4 : Actions
        self.actions =  [
            {
              "who": "driver",
              "type": "debit",
              "amount": driver_debit
            },
            {
              "who": "owner",
              "type": "credit",
              "amount": owner_fee
            },
            {
              "who": "insurance",
              "type": "credit",
              "amount": insurance_fee
            },
            {
              "who": "assistance",
              "type": "credit",
              "amount": assistance_fee
            },
            {
              "who": "drivy",
              "type": "credit",
              "amount": drivy_fee
            }
          ]
        

In [16]:
my_rent = Rental(1, veh, "2015-07-3", "2015-07-14", 1000)

In [17]:
my_rent = Rental(1, veh, "2015-03-31", "2015-04-01", 300)

In [18]:
my_rent.actions

[{'who': 'driver', 'type': 'debit', 'amount': 6800.0},
 {'who': 'owner', 'type': 'credit', 'amount': 4760.0},
 {'who': 'insurance', 'type': 'credit', 'amount': 1020.0},
 {'who': 'assistance', 'type': 'credit', 'amount': 200},
 {'who': 'drivy', 'type': 'credit', 'amount': 820.0}]

## <u>Level 5</u>
***

Some drivers want to be able to buy additionnal features after their booking.

Here are the possible options:

    - GPS: 5€/day, all the money goes to the owner
    - Baby Seat: 2€/day, all the money goes to the owner
    - Additional Insurance: 10€/day, all the money goes to Getaround

This is the final level, now would be a good time to tidy up your code and do a last round of refactoring :)

In [19]:
import datetime

class Rental():
    
    def __init__(self, rent_id, vehicle, start_date, end_date, distance):
        
        self.rent_id = rent_id
        self.vehicle = vehicle
        self.start_date = start_date
        self.end_date = end_date
        self.distance = distance
        self.commission = dict()
        
        #Calculate number of days
        diff_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.datetime.strptime(start_date, '%Y-%m-%d') #Calculate the number of days
        self.nb_days = diff_date.days + 1
        self.base_price = self.vehicle.get_rental_price(self.nb_days, distance) #Set the rental price 
     
    
        #Share rental revenues between car owner, the company, the insurance & the roadside assistance

        # For Assistance_fee, it is say in the instructions that it is 1€ per day of reservation
        # but to match the output file expectations,
        # i have to multiply it by 100
                
          # for drivy_fee, we assume the commission will
          # always be positive, if not it will count
          # as a company loss or debt
        
        self.commission = {
            "insurance_fee": self.base_price * 0.3 * 0.5,
            "assistance_fee": self.nb_days * 100,
            "drivy_fee": self.base_price * 0.3 - self.base_price * 0.3 * 0.5 - self.nb_days * 100 
         }
        
        
        #For Level 5 : Additional options
        self.options = []
        self.gps_fee = 0
        self.baby_seat_fee = 0
        self.add_insurance_fee = 0
        
        
        
    #Level 4 : Actions
        driver_debit = self.base_price 
        owner_fee = self.base_price * 0.7
        insurance_fee = self.commission["insurance_fee"]
        assistance_fee = self.commission["assistance_fee"]
        drivy_fee = self.commission["drivy_fee"]
        
        self.set_actions(driver_debit = driver_debit,
                         owner_fee = owner_fee,
                         insurance_fee = insurance_fee,
                         assistance_fee = assistance_fee,
                         drivy_fee = drivy_fee)
        
    
    def set_actions(self, driver_debit, owner_fee, insurance_fee,
                    assistance_fee, drivy_fee):
        
        #Level 4 : Actions
        self.actions =  [
            {
              "who": "driver",
              "type": "debit",
              "amount": driver_debit
            },
            {
              "who": "owner",
              "type": "credit",
              "amount": owner_fee
            },
            {
              "who": "insurance",
              "type": "credit",
              "amount": insurance_fee
            },
            {
              "who": "assistance",
              "type": "credit",
              "amount": assistance_fee
            },
            {
              "who": "drivy",
              "type": "credit",
              "amount": drivy_fee
            }
          ]
    
    #Level5 : Additional options
    def add_options(self, options=[]):
        
        self.options = options
        
        #Level 5 Additional options
        self.gps_fee = 500 * self.nb_days if("gps" in self.options) else 0
        self.baby_seat_fee = 200 * self.nb_days if("baby_seat" in self.options) else 0
        self.add_insurance_fee = 1000 * self.nb_days if("additional_insurance" in self.options) else 0
        
        driver_debit = self.base_price + self.gps_fee + self.baby_seat_fee + self.add_insurance_fee 
        owner_fee = self.base_price * 0.7 + self.gps_fee + self.baby_seat_fee
        insurance_fee = self.commission["insurance_fee"]
        assistance_fee = self.commission["assistance_fee"]
        drivy_fee = self.commission["drivy_fee"] + self.add_insurance_fee
    
        
        self.set_actions(driver_debit = driver_debit,
                         owner_fee = owner_fee,
                         insurance_fee = insurance_fee,
                         assistance_fee = assistance_fee,
                         drivy_fee = drivy_fee)
        
        
        

In [20]:
options=["additional_insurance"]

In [21]:
my_rent = Rental(1, veh, "2015-03-31", "2015-04-01", 300)

In [22]:
my_rent.actions

[{'who': 'driver', 'type': 'debit', 'amount': 6800.0},
 {'who': 'owner', 'type': 'credit', 'amount': 4760.0},
 {'who': 'insurance', 'type': 'credit', 'amount': 1020.0},
 {'who': 'assistance', 'type': 'credit', 'amount': 200},
 {'who': 'drivy', 'type': 'credit', 'amount': 820.0}]

In [23]:
my_rent.add_options(options)

In [24]:
my_rent.actions

[{'who': 'driver', 'type': 'debit', 'amount': 8800.0},
 {'who': 'owner', 'type': 'credit', 'amount': 4760.0},
 {'who': 'insurance', 'type': 'credit', 'amount': 1020.0},
 {'who': 'assistance', 'type': 'credit', 'amount': 200},
 {'who': 'drivy', 'type': 'credit', 'amount': 2820.0}]

#

## <center><u> Test : </u></center>

In [25]:
import json

In [26]:
#Load cars
def load_cars(input_data_cars):
    list_cars = []

    for car in input_data_cars:
        veh = Vehicle(car['id'], car['price_per_day'], car['price_per_km'])
        list_cars.append(veh)
    
    return list_cars

In [27]:
#Load rentals
def load_rentals(input_data, list_cars):
    
    #Load rentals
    list_rentals = []

    for rental in input_data['rentals']:

        #Get rental vehicle
        for car in list_cars:
            if(car.car_id == rental['car_id']):
                rent_veh = car 

        rent = Rental(rental['id'], rent_veh, rental['start_date'], rental['end_date'], rental['distance'])
        
        #Load options
        if "options" in input_data.keys():
            rental_options = []
            for option in input_data["options"]:
                if(option["rental_id"] == rent.rent_id):
                    rental_options.append(option["type"])
            
            rent.add_options(rental_options)
        
        list_rentals.append(rent)
        
    return list_rentals

In [28]:
def load_data(filename):
    
    with open(filename) as inputfile:
        input_data = json.load(inputfile)
    
    list_cars = load_cars(input_data["cars"])
    list_rentals = load_rentals(input_data, list_cars)
    
    return list_cars, list_rentals

In [29]:
def write_output(list_rentals, output_file):
    
    list_output_rentals = []

    for rental in list_rentals : 

        output_rental = {
            "id": rental.rent_id,
            "options": rental.options,
            "actions": rental.actions
        }

        list_output_rentals.append(output_rental)

    output = {
        "rentals" : list_output_rentals
    }

    with open(output_file, 'w') as fp:
        json.dump(output, fp)

In [30]:
import json

def test(input_file, output_name):
    list_cars, list_rentals = load_data(input_file)
    write_output(list_rentals, output_name)
    

In [ ]:
test("data/input.json", "data/output_2.json")